In [48]:
import pandas as pd
import zipfile
import os
import sys
from datetime import datetime
from datetime import timedelta
import numpy as np
pd.set_option('display.max_columns', 500)

In [49]:
# dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
vars_serg = ["estadoOrden","Ciudad","Departamento",
             "stado_producto","Marca","Pasillo","Categoria",
             "ProductoEnPromocion","cantidadVendida",
             "TotalPagado","Domicilio","PrecioFinal",
             "FechaCreaciónOrden"]

In [ ]:
import dask.dataframe as dd

In [ ]:
df_dask = dd.read_csv("../Data/MERQ_CONS",
                      usecols=vars_serg,
                      encoding='latin1',
                      assume_missing=True,
                     dtype={'estadoOrden' : str,
                            'TotalPagado' : np.float64,
                            "Domicilio" : str,
                            "FechaCreaciónOrden" : str,
                            "FechaProgramaciónOrden" : str,
                            "FechadeEntregaOrden" : str,
                            "Ciudad" : str,
                            "fechacreacionProducto" : str,
                            "stado_producto" : str,
                            "Marca" : str,
                            "Departamento" : str,
                            "Pasillo" : str,
                            "Categoria" : str,
                            "ProductoEnPromocion" : np.float64,
                            "cantidadVendida" : np.float64,
                            "PrecioFinal": np.float64})

In [ ]:
df_dask = df_dask.dropna()
df_dask.set_index("FechaCreaciónOrden")
df_dask["FechaCreaciónOrden"] = df_dask["FechaCreaciónOrden"].map_partitions(pd.to_datetime,
                                                                             format='%Y-%m-%d',
                                                                             meta = ('datetime64[ns]'))encoding='latin1'

In [ ]:
df_dask.head()

In [69]:
vars_group_b = ["FechaCreaciónOrden",'month_year',
                "Ciudad","Departamento",
                "estadoOrden","stado_producto",
                "Marca","Pasillo","Categoria",
                "ProductoEnPromocion"]

In [ ]:
%time agg = df_dask.groupby(vars_group_b).sum()

In [51]:
df_p = pd.read_csv("../Data/MERQ_CONS",
                      usecols=vars_serg,
                      encoding='latin1',
                     dtype={'estadoOrden' : str,
                            'TotalPagado' : np.float64,
                            "Domicilio" : str,
                            "FechaCreaciónOrden" : str,
                            "FechaProgramaciónOrden" : str,
                            "FechadeEntregaOrden" : str,
                            "Ciudad" : str,
                            "fechacreacionProducto" : str,
                            "stado_producto" : str,
                            "Marca" : str,
                            "Departamento" : str,
                            "Pasillo" : str,
                            "Categoria" : str,
                            "ProductoEnPromocion" : np.float64,
                            "cantidadVendida" : np.float64,
                            "PrecioFinal": np.float64})

In [52]:
df_p = df_p.dropna()

In [59]:
df_p["cuenta_gr"] = [1]*df_p.shape[0]

In [54]:
df_p['FechaCreaciónOrden'] = pd.to_datetime(df_p.FechaCreaciónOrden)

In [64]:
df_p['month_year'] = df_p.FechaCreaciónOrden.apply(lambda x: "%d-%d" % (x.year,x.month))
df_p['month_year'] = pd.to_datetime(df_p.month_year)

In [70]:
%time agg = df_p.groupby(vars_group_b,as_index=False).sum()

CPU times: user 3.91 s, sys: 569 ms, total: 4.48 s
Wall time: 3.82 s


In [57]:
agg.shape

(638906, 13)

In [71]:
agg.to_csv("../Data/AGG_MERQUEO.csv",encoding='latin1',index=False)

## Agregado sub set

In [ ]:
len(dict(enumerate(df['month_year'].unique())).keys())

In [67]:
%time df = pd.read_csv("../Data/AGG_MERQUEO.csv", encoding='latin1')

CPU times: user 1.31 s, sys: 8.29 ms, total: 1.32 s
Wall time: 1.21 s


In [4]:
df['FechaCreaciónOrden'] = pd.to_datetime(df.FechaCreaciónOrden)
df['month_year'] = pd.to_datetime(df.FechaCreaciónOrden)

In [5]:
df.set_index("FechaCreaciónOrden",inplace=True,drop=False)

In [68]:
df.columns

Index(['FechaCreaciónOrden', 'Ciudad', 'Departamento', 'estadoOrden',
       'stado_producto', 'Marca', 'Pasillo', 'Categoria',
       'ProductoEnPromocion', 'TotalPagado', 'cantidadVendida', 'PrecioFinal',
       'cueta_gr', 'cuenta_gr'],
      dtype='object')

In [ ]:
df_sb = df[df.Ciudad.str.contains("Bogotá")][['cantidadVendida']]
df_sb = df_sb.resample('D').sum()

In [ ]:
[ "%s %dM" % (element[0],element[1]/1000000000) for element in zip(df['2017-2':'2018-02'].groupby("Ciudad").sum()[["TotalPagado"]].index,df['2017-2':'2018-02'].groupby("Ciudad").sum()[["TotalPagado"]].values)]

In [7]:
difagg = df['2017-2':'2018-02'].groupby("Ciudad").sum()[["TotalPagado"]]
" | ".join(["%s %dM" % (e[0],e[1]/1000000000) for e in zip(difagg.index,difagg.values)])

'Bogotá 506M | Medellín 139M'

In [9]:
difagg = df.groupby("estadoOrden").sum()[["cueta_gr"]]/\
                df.groupby("estadoOrden").sum()[["cueta_gr"]].sum()
" | ".join([e for e in zip(difagg.index,difagg.values)])

TypeError: sequence item 0: expected str instance, tuple found

In [10]:
lista_estados_ordenes = list(zip(difagg.index,difagg.values))

In [13]:
["{}%".format(str(np.round(e[1],3)[0]*100)) for e in lista_estados_ordenes]

['0.0%', '12.3%', '87.3%', '0.0%', '0.3%', '0.0%']

In [14]:
[e for e in lista_estados_ordenes]

[('Alistado', array([1.95198168e-06])),
 ('Cancelled', array([0.12345612])),
 ('Delivered', array([0.87344522])),
 ('Dispatched', array([0.00014054])),
 ('Initiated', array([0.00289978])),
 ('Validation', array([5.6390582e-05]))]

In [17]:
indicador = lista_estados_ordenes[2]
"{}%".format(str(np.round(indicador[1],3)[0]*100))

'87.3%'

In [27]:
%time df[df.Ciudad.str.contains("Bogotá") & df.Pasillo.str.contains("Aceite") & df.Categoria.str.contains("")].shape

CPU times: user 1.08 s, sys: 3.7 ms, total: 1.08 s
Wall time: 1.08 s


(6460, 14)

,Pasillo,Categoria,ProductoEnPromocion,TotalPagado,cantidadVendida,PrecioFinal,cueta_gr
0,Accesorios,Gimnasio,45.0,5.688788e+06,159.0,3794414.0,158
1,Aceites y aderezos,Aceite,1100.0,1.064208e+10,68177.0,576726338.0,62008
2,Aceites y aderezos,Aceite de Oliva,144.0,1.010231e+09,6209.0,68259535.0,5796
3,Aceites y aderezos,Vinagre,30.0,2.144118e+09,13812.0,19228327.0,10411
4,Aceites y aderezos,Vinagreta,0.0,2.494718e+08,811.0,2417801.0,788
5,Aguardiente,Aguardiente,15.0,1.573591e+08,1120.0,12878509.0,537
6,Aguas,Agua,190.0,2.317387e+09,45455.0,28783678.0,17052
7,Aguas,Agua de coco,1.0,9.358460e+05,4.0,241790.0,4
8,Aguas,Agua saborizada,59.0,9.917226e+08,15345.0,11628242.0,5227
9,Alimento para bebé,Alimento lácteo,61.0,7.995091e+08,11608.0,22887897.0,4755
